# cuSlide vs cuSlide2 Performance Comparison

This notebook compares performance between cuslide (legacy) and cuslide2 (nvImageCodec-based) plugins for cuCIM.

## Key Features Compared
- **Single Region Decoding**: CPU and GPU performance
- **Batch Decoding**: New in cuslide2 using nvImageCodec v0.7.0+ API
- **Multi-Resolution Support**: Reading at different pyramid levels
- **Format Support**: Aperio SVS and Philips TIFF

## Requirements
- cuCIM with cuslide2 plugin built
- nvImageCodec v0.7.0+
- CuPy (for GPU arrays)
- Test images (Aperio SVS or Philips TIFF)


In [ ]:
# Core imports
import json
import os
import sys
import time
import tempfile
from pathlib import Path
from contextlib import contextmanager

import numpy as np
import matplotlib.pyplot as plt

# Try to import CuPy for GPU support
try:
    import cupy as cp
    HAS_CUPY = True
except ImportError:
    HAS_CUPY = False
    print("⚠️  CuPy not available - GPU tests will be skipped")

print(f"NumPy version: {np.__version__}")
if HAS_CUPY:
    print(f"CuPy version: {cp.__version__}")


## 1. Setup Plugin Configuration

We need to configure cuCIM to use the appropriate plugin (cuslide or cuslide2).


In [ ]:
import cucim
from cucim import CuImage
from cucim.clara import _set_plugin_root

print(f"cuCIM version: {cucim.__version__}")

# Get repository root
REPO_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
print(f"Repository root: {REPO_ROOT}")

# Plugin paths
CUSLIDE_PATH = REPO_ROOT / "cpp" / "plugins" / "cucim.kit.cuslide" / "build-release" / "lib"
CUSLIDE2_PATH = REPO_ROOT / "cpp" / "plugins" / "cucim.kit.cuslide2" / "build-release" / "lib"

print(f"cuslide path: {CUSLIDE_PATH} (exists: {CUSLIDE_PATH.exists()})")
print(f"cuslide2 path: {CUSLIDE2_PATH} (exists: {CUSLIDE2_PATH.exists()})")


In [ ]:
@contextmanager
def use_plugin(plugin_name: str):
    """Context manager to switch between cuslide and cuslide2 plugins."""
    if plugin_name == "cuslide":
        plugin_path = CUSLIDE_PATH
        so_name = f"cucim.kit.cuslide@{cucim.__version__}.so"
    elif plugin_name == "cuslide2":
        plugin_path = CUSLIDE2_PATH
        so_name = f"cucim.kit.cuslide2@{cucim.__version__}.so"
    else:
        raise ValueError(f"Unknown plugin: {plugin_name}")
    
    if not plugin_path.exists():
        raise FileNotFoundError(f"Plugin not found at {plugin_path}")
    
    # Create config
    config = {
        "plugin": {
            "names": [so_name]
        }
    }
    
    config_file = tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False)
    json.dump(config, config_file)
    config_file.close()
    
    old_config = os.environ.get("CUCIM_CONFIG_PATH")
    os.environ["CUCIM_CONFIG_PATH"] = config_file.name
    _set_plugin_root(str(plugin_path))
    
    try:
        yield
    finally:
        if old_config:
            os.environ["CUCIM_CONFIG_PATH"] = old_config
        os.unlink(config_file.name)


## 2. Download Test Images

We'll use test images from OpenSlide's test data repository.


In [ ]:
import urllib.request

TEST_DATA_DIR = Path("/tmp/cucim_test_data")
TEST_DATA_DIR.mkdir(exist_ok=True)

# Test files configuration
TEST_FILES = {
    "aperio_small": {
        "url": "https://openslide.cs.cmu.edu/download/openslide-testdata/Aperio/CMU-1-Small-Region.svs",
        "filename": "CMU-1-Small-Region.svs",
        "description": "Aperio SVS - Small region (2MB)",
        "format": "Aperio SVS"
    },
    # Uncomment for larger files:
    # "aperio_full": {
    #     "url": "https://openslide.cs.cmu.edu/download/openslide-testdata/Aperio/CMU-1.svs",
    #     "filename": "CMU-1.svs",
    #     "description": "Aperio SVS - Full slide (177MB)",
    #     "format": "Aperio SVS"
    # },
    # "philips": {
    #     "url": "https://openslide.cs.cmu.edu/download/openslide-testdata/Philips-TIFF/Philips-1.tiff",
    #     "filename": "Philips-1.tiff",
    #     "description": "Philips TIFF - Lymph node (311MB)",
    #     "format": "Philips TIFF"
    # }
}

def download_test_file(file_key: str):
    """Download a test file if not already present."""
    file_info = TEST_FILES[file_key]
    local_path = TEST_DATA_DIR / file_info["filename"]
    
    if local_path.exists():
        print(f"✅ {file_info['filename']} already exists")
        return str(local_path)
    
    print(f"📥 Downloading {file_info['description']}...")
    print(f"   URL: {file_info['url']}")
    
    urllib.request.urlretrieve(file_info["url"], local_path)
    print(f"✅ Downloaded: {local_path}")
    
    return str(local_path)

# Download the small test file
test_svs_path = download_test_file("aperio_small")
print(f"\nTest file: {test_svs_path}")


## 4. Single Region Read Performance

Compare single tile/region read performance between cuslide and cuslide2.


In [ ]:
def benchmark_single_read(image_path: str, plugin: str, device: str, 
                          size=(512, 512), level=0, iterations=10):
    """Benchmark single region read."""
    result = BenchmarkResult(f"{plugin} ({device})")
    
    with use_plugin(plugin):
        img = CuImage(image_path)
        
        # Warmup
        _ = img.read_region((0, 0), size, level, device=device)
        
        # Benchmark
        for _ in range(iterations):
            start = time.perf_counter()
            region = img.read_region((0, 0), size, level, device=device)
            if device == "cuda" and HAS_CUPY:
                cp.cuda.Stream.null.synchronize()  # Ensure GPU work is done
            elapsed = time.perf_counter() - start
            result.add_time(elapsed)
        
        result.metadata['shape'] = region.shape
        result.metadata['device'] = region.device
    
    return result

print("Single read benchmark function defined ✅")


In [ ]:
print("\n" + "="*60)
print("Single Region Read Benchmark (512x512)")
print("="*60)

single_read_results = {}

# Test cuslide2 (nvImageCodec)
if CUSLIDE2_PATH.exists():
    try:
        result = benchmark_single_read(test_svs_path, "cuslide2", "cpu")
        single_read_results["cuslide2 (CPU)"] = result
        print(f"✅ {result}")
    except Exception as e:
        print(f"❌ cuslide2 CPU failed: {e}")
    
    if HAS_CUPY:
        try:
            result = benchmark_single_read(test_svs_path, "cuslide2", "cuda")
            single_read_results["cuslide2 (GPU)"] = result
            print(f"✅ {result}")
        except Exception as e:
            print(f"⚠️ cuslide2 GPU failed: {e}")
else:
    print("⚠️ cuslide2 not built")

# Test cuslide (legacy)
if CUSLIDE_PATH.exists():
    try:
        result = benchmark_single_read(test_svs_path, "cuslide", "cpu")
        single_read_results["cuslide (CPU)"] = result
        print(f"✅ {result}")
    except Exception as e:
        print(f"❌ cuslide CPU failed: {e}")
else:
    print("⚠️ cuslide not built")

if single_read_results:
    plot_comparison(single_read_results, "Single Region Read (512x512)")


## 5. Batch Decoding Performance (cuslide2 only)

This is the new batch decoding feature in cuslide2 using nvImageCodec v0.7.0+ API.
It uses `get_sub_code_stream()` for multiple ROIs and a single `decoder.decode()` call.


In [ ]:
def benchmark_batch_read(image_path: str, num_locations: int, device: str,
                         size=(256, 256), num_workers=4, batch_size=8,
                         iterations=5):
    """Benchmark batch region read (cuslide2 only)."""
    result = BenchmarkResult(f"batch={num_locations} ({device})")
    
    with use_plugin("cuslide2"):
        img = CuImage(image_path)
        width, height = img.size("XY")
        
        # Generate random locations
        np.random.seed(42)
        locations = [
            (np.random.randint(0, max(1, width - size[0])),
             np.random.randint(0, max(1, height - size[1])))
            for _ in range(num_locations)
        ]
        
        # Warmup
        gen = img.read_region(locations, size, 0, num_workers=num_workers, 
                              batch_size=batch_size, device=device)
        _ = list(gen)
        
        # Benchmark
        for _ in range(iterations):
            start = time.perf_counter()
            gen = img.read_region(locations, size, 0, num_workers=num_workers,
                                  batch_size=batch_size, device=device)
            results = list(gen)
            if device == "cuda" and HAS_CUPY:
                cp.cuda.Stream.null.synchronize()
            elapsed = time.perf_counter() - start
            result.add_time(elapsed)
        
        result.metadata['num_regions'] = len(results)
        result.metadata['throughput'] = num_locations / result.mean  # regions/sec
    
    return result

print("Batch read benchmark function defined ✅")


In [ ]:
print("\n" + "="*60)
print("Batch Decoding Benchmark (cuslide2 with nvImageCodec)")
print("="*60)

if CUSLIDE2_PATH.exists():
    batch_results = {}
    
    for num_locs in [4, 16, 64, 256]:
        try:
            result = benchmark_batch_read(test_svs_path, num_locs, "cpu")
            batch_results[f"{num_locs} regions"] = result
            throughput = result.metadata.get('throughput', 0)
            print(f"✅ {result} | Throughput: {throughput:.0f} regions/sec")
        except Exception as e:
            print(f"⚠️ Batch {num_locs} failed: {e}")
    
    if batch_results:
        plot_comparison(batch_results, "Batch Decoding - Varying Region Count (256x256 tiles)")
else:
    print("⚠️ cuslide2 not built - batch decoding unavailable")


## 6. Sequential vs Batch Comparison

Compare sequential single-region reads against batch decoding.


In [ ]:
print("\n" + "="*60)
print("Sequential vs Batch Decoding Comparison")
print("="*60)

NUM_REGIONS = 32
TILE_SIZE = (256, 256)

if CUSLIDE2_PATH.exists():
    with use_plugin("cuslide2"):
        img = CuImage(test_svs_path)
        width, height = img.size("XY")
        
        np.random.seed(42)
        locations = [
            (np.random.randint(0, max(1, width - TILE_SIZE[0])),
             np.random.randint(0, max(1, height - TILE_SIZE[1])))
            for _ in range(NUM_REGIONS)
        ]
        
        comparison_results = {}
        
        # Sequential (single-threaded)
        print(f"\nBenchmarking {NUM_REGIONS} regions...")
        
        seq_result = BenchmarkResult("Sequential")
        for _ in range(3):  # 3 iterations
            start = time.perf_counter()
            for loc in locations:
                _ = img.read_region(loc, TILE_SIZE, 0)
            seq_result.add_time(time.perf_counter() - start)
        comparison_results["Sequential"] = seq_result
        print(f"✅ {seq_result}")
        
        # Batch (multi-threaded)
        batch_result = BenchmarkResult("Batch (4 workers)")
        for _ in range(3):
            start = time.perf_counter()
            _ = list(img.read_region(locations, TILE_SIZE, 0, num_workers=4))
            batch_result.add_time(time.perf_counter() - start)
        comparison_results["Batch (4 workers)"] = batch_result
        print(f"✅ {batch_result}")
        
        # Calculate speedup
        speedup = seq_result.mean / batch_result.mean
        print(f"\n🚀 Batch decoding speedup: {speedup:.2f}x faster")
        
        plot_comparison(comparison_results, f"Sequential vs Batch ({NUM_REGIONS} regions)")
else:
    print("⚠️ cuslide2 not built")


## 7. GPU Batch Decoding (JPEG only)

Test batch decoding with GPU output. Note: GPU output is only supported for JPEG compression.


In [ ]:
if HAS_CUPY and CUSLIDE2_PATH.exists():
    print("\n" + "="*60)
    print("GPU Batch Decoding (JPEG compression only)")
    print("="*60)
    
    try:
        with use_plugin("cuslide2"):
            img = CuImage(test_svs_path)
            width, height = img.size("XY")
            
            np.random.seed(42)
            locations = [
                (np.random.randint(0, max(1, width - 256)),
                 np.random.randint(0, max(1, height - 256)))
                for _ in range(32)
            ]
            
            gpu_results = {}
            
            # CPU baseline
            cpu_result = BenchmarkResult("CPU")
            for _ in range(5):
                start = time.perf_counter()
                _ = list(img.read_region(locations, (256, 256), 0, num_workers=4, device="cpu"))
                cpu_result.add_time(time.perf_counter() - start)
            gpu_results["CPU"] = cpu_result
            print(f"✅ {cpu_result}")
            
            # GPU
            gpu_result = BenchmarkResult("GPU (CUDA)")
            for _ in range(5):
                start = time.perf_counter()
                results = list(img.read_region(locations, (256, 256), 0, num_workers=4, device="cuda"))
                cp.cuda.Stream.null.synchronize()
                gpu_result.add_time(time.perf_counter() - start)
            gpu_results["GPU (CUDA)"] = gpu_result
            print(f"✅ {gpu_result}")
            
            # Verify GPU arrays
            if results:
                arr = cp.asarray(results[0])
                print(f"\n📊 GPU array info:")
                print(f"   Shape: {arr.shape}")
                print(f"   Device: cuda:{arr.device.id}")
                print(f"   Dtype: {arr.dtype}")
            
            speedup = cpu_result.mean / gpu_result.mean
            print(f"\n🚀 GPU speedup: {speedup:.2f}x")
            
            plot_comparison(gpu_results, "CPU vs GPU Batch Decoding (32 regions)")
    except Exception as e:
        print(f"⚠️ GPU batch decoding failed: {e}")
        print("   This may be expected if the test file uses non-JPEG compression.")
else:
    print("⚠️ GPU batch decoding requires CuPy and cuslide2")


## 8. Summary


In [ ]:
print("\n" + "="*60)
print("📊 Performance Summary")
print("="*60)

print("""
cuslide2 (nvImageCodec v0.7.0+) Features:

✅ Batch ROI Decoding
   - Uses nvimgcodecCodeStreamGetSubCodeStream() for multiple ROIs
   - Single nvimgcodecDecoderDecode() call for all regions
   - Significantly faster than sequential reads

✅ Multi-threaded Processing
   - ThreadBatchDataLoader with configurable workers
   - Prefetch and batch size parameters
   - Shuffle support for training workflows

✅ GPU Acceleration (JPEG)
   - Direct GPU decoding for JPEG-compressed tiles
   - Zero-copy output to CUDA memory
   - Integration with CuPy arrays

✅ Format Support
   - Aperio SVS (JPEG, JPEG 2000)
   - Philips TIFF (with metadata parsing)
   - Generic TIFF (JPEG, Deflate, Raw)
""")

print("\n✅ Benchmark complete!")


## Running the Unit Tests

To run the batch decoding unit tests from the command line:

```bash
# From the cucim directory:
cd cucim

# Run all batch decoding tests
pytest python/cucim/tests/unit/clara/test_batch_decoding.py -v

# Run specific test categories
pytest python/cucim/tests/unit/clara/test_batch_decoding.py::TestBatchDecoding -v
pytest python/cucim/tests/unit/clara/test_batch_decoding.py::TestBatchDecodingCUDA -v
pytest python/cucim/tests/unit/clara/test_batch_decoding.py::TestBatchDecodingPerformance -v
```

Test categories:
- **TestBatchDecoding**: CPU batch decoding tests (21 tests)
- **TestBatchDecodingCUDA**: GPU batch decoding tests for JPEG images (2 tests)
- **TestBatchDecodingPerformance**: Scaling tests with varying locations and workers (24 tests)

## Usage Example

```python
from cucim import CuImage
import numpy as np

img = CuImage("slide.svs")

# Batch decode 1000 tiles with 4 workers
locations = [(x, y) for x, y in tile_coordinates]
for batch in img.read_region(
    locations, size=(256, 256), level=0,
    num_workers=4, batch_size=32, device="cuda"
):
    # Process batch on GPU
    arr = cp.asarray(batch)
    # ... your processing code ...
```


In [ ]:
class BenchmarkResult:
    """Store benchmark results."""
    def __init__(self, name: str):
        self.name = name
        self.times = []
        self.metadata = {}
    
    def add_time(self, t: float):
        self.times.append(t)
    
    @property
    def mean(self):
        return np.mean(self.times) if self.times else 0
    
    @property
    def std(self):
        return np.std(self.times) if len(self.times) > 1 else 0
    
    @property
    def min(self):
        return np.min(self.times) if self.times else 0
    
    def __repr__(self):
        return f"{self.name}: {self.mean*1000:.2f} ± {self.std*1000:.2f} ms (min: {self.min*1000:.2f} ms)"


def plot_comparison(results: dict, title: str, ylabel: str = "Time (ms)"):
    """Plot benchmark comparison."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    names = list(results.keys())
    means = [r.mean * 1000 for r in results.values()]
    stds = [r.std * 1000 for r in results.values()]
    
    colors = ['#2ecc71', '#3498db', '#e74c3c', '#9b59b6', '#f39c12']
    bars = ax.bar(names, means, yerr=stds, capsize=5, color=colors[:len(names)], alpha=0.8)
    
    ax.set_ylabel(ylabel, fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, mean in zip(bars, means):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(stds)*0.1,
                f'{mean:.1f}ms', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    return fig
